In [11]:
import pandas as pd
import qiime2 as q2
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
data_dir = 'data'
source = 'data/raw'
! mkdir -p data data/raw


Upload the files from Rodrigo to the raw directory

## Extract compressed files from .tar.gz archives

In [7]:
import tarfile
import os

####Run only once

#for filename in os.listdir(source):
#    if filename.endswith(".tar.gz"):
#        filepath = os.path.join(source, filename)
#        foldername = filename[:-7]  # remove .tar.gz
#        outdir = os.path.join(data_dir, foldername)
#        os.makedirs(outdir, exist_ok=True)
#        with tarfile.open(filepath, "r:gz") as tar:
#            tar.extractall(path=outdir)


In [12]:
import os
import pandas as pd

data_dir = "data"
source = "data/raw"

# load metadata
metadata_df = pd.read_csv(f"{source}/merged_metadata_filtered.tsv", sep="\t", index_col=0)

records = []

for technique in os.listdir(source):
    tech_path = os.path.join(source, technique)
    if not os.path.isdir(tech_path):
        continue
    for sample_id in os.listdir(tech_path):
        sample_path = os.path.join(tech_path, sample_id)
        if not os.path.isdir(sample_path):
            continue
        for f in os.listdir(sample_path):
            if f.endswith((".fa", ".fasta")):
                abs_path = os.path.abspath(os.path.join(sample_path, f))
                if sample_id in metadata_df.index:
                    records.append((sample_id, abs_path))

# build dataframe
manifest_df = pd.DataFrame.from_records(records, columns=["sample-id", "absolute-filepath"])
manifest_df.set_index("sample-id", inplace=True)

# export MANIFEST
manifest_df.to_csv(f"{data_dir}/MANIFEST", sep="\t")


In [14]:
######create Merged_df for exploration######
metadata_df['sample_id'] = metadata_df.index
merged_df=pd.merge(metadata_df, manifest_df, how = 'right', left_on = 'sample_ID', right_on = 'sample-id')
merged_df.to_csv(f"{data_dir}/metadata_merged.csv", index=False)